In [1]:
import urllib.request, urllib.parse, urllib.error
import ssl

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
import lxml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.7.1-py36_1 --> 4.8.2-py36_0 conda-forge


beautifulsoup4-4.8.2 | 157 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'lxml')

In [4]:
columns_names = []

headers = soup.tbody.tr.text.split('\n')

for name in headers:
    if len(name) > 1:
        columns_names.append(name)

In [5]:
neighborhoods = pd.DataFrame(columns=columns_names)

tab = soup.tbody.text.split('\n\n\n')[1:]

for n in zip(tab):
    postcode = str(n).split('\\n')[0][2:]
    borough = str(n).split('\\n')[1]
    neighbourhood = str(n).split('\\n')[2][:-3]

    if borough == 'Not assigned': continue
    if neighbourhood == 'Not assigned':
        neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighbourhood': borough,
                                          'Postcode': postcode}, ignore_index=True)
    else:
        neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighbourhood': neighbourhood,
                                          'Postcode': postcode}, ignore_index=True)      


In [6]:
neighborhoods['Postcode'].describe()

count     210
unique    103
top       M8Y
freq        8
Name: Postcode, dtype: object

In [7]:
neighborhoods = neighborhoods.groupby('Postcode').agg({'Borough':'first',  'Neighbourhood': ', '.join }).reset_index()

In [8]:
neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
neighborhoods.shape

(103, 3)

End of the first task.
The BeautifulSoup4, pandas, urllib, ssl and lxml package have been used. 

In [13]:
!conda install -c conda-forge geocoder --yes
import geocoder

Solving environment: done

# All requested packages already installed.



In [14]:
Latitude = []
Longitude = [] 

In [15]:
for index, row in neighborhoods.iterrows():
    Latitude.append(geocoder.arcgis('Toronto, ' + row[0]).lat)
    Longitude.append(geocoder.arcgis('Toronto, ' + row[0]).lng)

neighborhoods['Latitude'] = Latitude
neighborhoods['Longitude'] = Longitude

In [16]:
neighborhoods.shape

(103, 5)

In [18]:
neighborhoods.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967


End of the second task. The ArcGIS through Gegoder package have been used.